In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

In [2]:
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [3]:
sortedFeatureCorr = pd.read_csv("dataset/corr.csv")
data = pd.read_csv("dataset/preprocessed.csv")

In [4]:
# time stamp will add

In [5]:
first_feature_count =           22
last_feature_count =            21

In [6]:
a = sortedFeatureCorr.iloc[:, 0].values

In [7]:
print(a.shape)

(43,)


In [8]:
for i in range(first_feature_count , sortedFeatureCorr.shape[0] - last_feature_count):
    data = data.drop(a[i], axis=1)
data=data.drop(list(data.columns)[0], axis=1)


In [9]:
data_npWNan = data.iloc[:, :].values

data_np = data_npWNan[data_npWNan[:,-1]!= -1 ]
unique_elements, counts_elements = np.unique(data_np[:,-1], return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[     1      2      3      4      5]
 [ 78815  87257 136412 183844 275766]]


In [10]:
print("feature " + str(data_np.shape))


feature (762094, 43)


In [11]:
feature = data_np[:, :-1]
# target = data_np[:, -1].reshape(data_np.shape[0], 1)
target = data_np[:, -1]


In [12]:
# for x in range(10):
#     print(target[x])

In [13]:
target -= 1
# for x in range(10):
#     print(target[x])

In [14]:
X_trainAndVal, X_test, y_trainAndVal, y_test = train_test_split(feature, target, test_size = 0.2, random_state = 0)

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X_trainAndVal, y_trainAndVal, test_size = 0.2, random_state = 0)

In [16]:
print((X_train ).shape)
print(( y_train).shape)


(487740, 42)
(487740,)


In [17]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [18]:
batch_size = 1000
epochs = 50
input_dim = first_feature_count + last_feature_count - 1
output_dim = 5
lr_rate = 0.001


In [19]:
class datasetLoad(Dataset):
    def __init__(self, features,labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    
        
    

In [20]:
# asdasd=1

In [21]:
# print(X_train[asdasd])

In [22]:
# print(X_val[asdasd])

In [23]:
# print(X_test[asdasd])

In [ ]:
X_train = datasetLoad(X_train, y_train)
X_val = datasetLoad(X_val, y_val)
X_test = datasetLoad(X_test, y_test)


In [ ]:
# print(X_train.__getitem__(asdasd))
# print(X_train.__len__())


In [ ]:
# print(X_val.__getitem__(asdasd))
# print(X_val.__len__())

In [ ]:
# print(X_test.__getitem__(asdasd))
# print(X_test.__len__())

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = X_train, batch_size = batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = X_val, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = X_test, batch_size = batch_size, shuffle=True)

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
    

In [ ]:
model = LogisticRegression(input_dim, output_dim)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
# sgd ya da adam kullanilabilir
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
print(len(X_train))

In [ ]:
import datetime
a = datetime.datetime.now().replace(microsecond=0)
for epoch in range(epochs):
    for i, (features, labels) in enumerate(train_loader):
        

#         print(type(features))
#         print(type(labels))
        features = Variable(features)
        labels = Variable(labels)
#         print(type(features))
#         print(type(labels))
#         print(features.shape)
#         print(labels.shape)

        optimizer.zero_grad()
        outputs = model(features.float())
#         print("output ",outputs.shape)
#         print("label ",labels.shape)
#         print("output ",outputs[0])
#         print("label ",labels[0])
        loss = criterion(outputs.float(), labels)
        loss.backward()
        optimizer.step()

        
        if i % (len(X_train)// (4 * batch_size))  == 0:
            # calculate Accuracy
            val_correct = 0
            val_total = 0
#             print("val")
#             x=0
#             for val_features, val_labels in test_loader:
#                 if(x>1):
#                     break
#                 x+=1
#                 print(val_features[0], "  " , val_labels[0])
            for val_features, val_labels in val_loader:
                val_features = Variable(val_features)
                val_outputs = model(val_features.float())
#                 if(i==0): 
#                     print("val")
#                     print(val_outputs)
                _, val_predicted = torch.max(val_outputs.data, 1)
                val_total+= val_labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                val_correct+= (val_predicted == val_labels).sum()
#             print("val")
#             print(val_outputs.shape)
            val_accuracy = 100 * val_correct / val_total
            print("val Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), val_accuracy))
        if i % (len(X_train)// (4 * batch_size))  == 0:
            # calculate Accuracy
            test_correct = 0
            test_total = 0
#             print("test")
#             x=0
#             for test_features, test_labels in test_loader:
#                 if(x>1):
#                     break
#                 x+=1
#                 print(test_features[0], "  " , test_labels[0])
            for test_features, test_labels in test_loader:
                test_features = Variable(test_features)
                test_outputs = model(test_features.float())
#                 if(i==0):
#                     print("test")
#                     print(test_outputs)
                
                _, test_predicted = torch.max(test_outputs.data, 1)
                test_total+= test_labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                test_correct+= (test_predicted == test_labels).sum()
            test_accuracy = 100 * test_correct / test_total
#             print("test")
#             print(test_outputs.shape)
            print("test Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), test_accuracy))

            


In [ ]:
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

In [25]:
# print(X_train.__getitem__(asdasd))
# print(X_train.__len__())


In [26]:
# print(X_val.__getitem__(asdasd))
# print(X_val.__len__())

In [27]:
# print(X_test.__getitem__(asdasd))
# print(X_test.__len__())

In [28]:
train_loader = torch.utils.data.DataLoader(dataset = X_train, batch_size = batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = X_val, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = X_test, batch_size = batch_size, shuffle=True)

In [29]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
    

In [30]:
model = LogisticRegression(input_dim, output_dim)

In [31]:
criterion = torch.nn.CrossEntropyLoss()


In [32]:
# sgd ya da adam kullanilabilir
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [33]:
print(len(X_train))

487740


In [34]:
import datetime
a = datetime.datetime.now().replace(microsecond=0)
for epoch in range(epochs):
    for i, (features, labels) in enumerate(train_loader):
        

#         print(type(features))
#         print(type(labels))
        features = Variable(features)
        labels = Variable(labels)
#         print(type(features))
#         print(type(labels))
#         print(features.shape)
#         print(labels.shape)

        optimizer.zero_grad()
        outputs = model(features.float())
#         print("output ",outputs.shape)
#         print("label ",labels.shape)
#         print("output ",outputs[0])
#         print("label ",labels[0])
        loss = criterion(outputs.float(), labels)
        loss.backward()
        optimizer.step()

        
        if i % (len(X_train)// (4 * batch_size))  == 0:
            # calculate Accuracy
            val_correct = 0
            val_total = 0
#             print("val")
#             x=0
#             for val_features, val_labels in test_loader:
#                 if(x>1):
#                     break
#                 x+=1
#                 print(val_features[0], "  " , val_labels[0])
            for val_features, val_labels in val_loader:
                val_features = Variable(val_features)
                val_outputs = model(val_features.float())
#                 if(i==0): 
#                     print("val")
#                     print(val_outputs)
                _, val_predicted = torch.max(val_outputs.data, 1)
                val_total+= val_labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                val_correct+= (val_predicted == val_labels).sum()
#             print("val")
#             print(val_outputs.shape)
            val_accuracy = 100 * val_correct / val_total
            print("val Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), val_accuracy))
        if i % (len(X_train)// (4 * batch_size))  == 0:
            # calculate Accuracy
            test_correct = 0
            test_total = 0
#             print("test")
#             x=0
#             for test_features, test_labels in test_loader:
#                 if(x>1):
#                     break
#                 x+=1
#                 print(test_features[0], "  " , test_labels[0])
            for test_features, test_labels in test_loader:
                test_features = Variable(test_features)
                test_outputs = model(test_features.float())
#                 if(i==0):
#                     print("test")
#                     print(test_outputs)
                
                _, test_predicted = torch.max(test_outputs.data, 1)
                test_total+= test_labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                test_correct+= (test_predicted == test_labels).sum()
            test_accuracy = 100 * test_correct / test_total
#             print("test")
#             print(test_outputs.shape)
            print("test Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), test_accuracy))

            


val Epoch: 0. Iteration: 0. Loss: 1.6612980365753174. Accuracy: 20.
test Epoch: 0. Iteration: 0. Loss: 1.6612980365753174. Accuracy: 20.
val Epoch: 0. Iteration: 121. Loss: 1.6268490552902222. Accuracy: 22.
test Epoch: 0. Iteration: 121. Loss: 1.6268490552902222. Accuracy: 22.
val Epoch: 0. Iteration: 242. Loss: 1.6288831233978271. Accuracy: 25.
test Epoch: 0. Iteration: 242. Loss: 1.6288831233978271. Accuracy: 25.
val Epoch: 0. Iteration: 363. Loss: 1.5850733518600464. Accuracy: 28.
test Epoch: 0. Iteration: 363. Loss: 1.5850733518600464. Accuracy: 28.
val Epoch: 0. Iteration: 484. Loss: 1.554494023323059. Accuracy: 31.
test Epoch: 0. Iteration: 484. Loss: 1.554494023323059. Accuracy: 31.
val Epoch: 1. Iteration: 0. Loss: 1.5513596534729004. Accuracy: 31.
test Epoch: 1. Iteration: 0. Loss: 1.5513596534729004. Accuracy: 31.
val Epoch: 1. Iteration: 121. Loss: 1.5703109502792358. Accuracy: 33.
test Epoch: 1. Iteration: 121. Loss: 1.5703109502792358. Accuracy: 33.
val Epoch: 1. Iteration

val Epoch: 11. Iteration: 484. Loss: 1.3586117029190063. Accuracy: 42.
test Epoch: 11. Iteration: 484. Loss: 1.3586117029190063. Accuracy: 42.
val Epoch: 12. Iteration: 0. Loss: 1.365082859992981. Accuracy: 42.
test Epoch: 12. Iteration: 0. Loss: 1.365082859992981. Accuracy: 42.
val Epoch: 12. Iteration: 121. Loss: 1.331795573234558. Accuracy: 43.
test Epoch: 12. Iteration: 121. Loss: 1.331795573234558. Accuracy: 42.
val Epoch: 12. Iteration: 242. Loss: 1.3464843034744263. Accuracy: 43.
test Epoch: 12. Iteration: 242. Loss: 1.3464843034744263. Accuracy: 42.
val Epoch: 12. Iteration: 363. Loss: 1.357080340385437. Accuracy: 43.
test Epoch: 12. Iteration: 363. Loss: 1.357080340385437. Accuracy: 42.
val Epoch: 12. Iteration: 484. Loss: 1.3508505821228027. Accuracy: 43.
test Epoch: 12. Iteration: 484. Loss: 1.3508505821228027. Accuracy: 42.
val Epoch: 13. Iteration: 0. Loss: 1.379441499710083. Accuracy: 43.
test Epoch: 13. Iteration: 0. Loss: 1.379441499710083. Accuracy: 42.
val Epoch: 13. 

val Epoch: 23. Iteration: 242. Loss: 1.2921334505081177. Accuracy: 43.
test Epoch: 23. Iteration: 242. Loss: 1.2921334505081177. Accuracy: 43.
val Epoch: 23. Iteration: 363. Loss: 1.3209654092788696. Accuracy: 43.
test Epoch: 23. Iteration: 363. Loss: 1.3209654092788696. Accuracy: 43.
val Epoch: 23. Iteration: 484. Loss: 1.3378890752792358. Accuracy: 43.
test Epoch: 23. Iteration: 484. Loss: 1.3378890752792358. Accuracy: 43.
val Epoch: 24. Iteration: 0. Loss: 1.3550090789794922. Accuracy: 43.
test Epoch: 24. Iteration: 0. Loss: 1.3550090789794922. Accuracy: 43.
val Epoch: 24. Iteration: 121. Loss: 1.3020809888839722. Accuracy: 43.
test Epoch: 24. Iteration: 121. Loss: 1.3020809888839722. Accuracy: 43.
val Epoch: 24. Iteration: 242. Loss: 1.3334583044052124. Accuracy: 43.
test Epoch: 24. Iteration: 242. Loss: 1.3334583044052124. Accuracy: 43.
val Epoch: 24. Iteration: 363. Loss: 1.3133807182312012. Accuracy: 43.
test Epoch: 24. Iteration: 363. Loss: 1.3133807182312012. Accuracy: 43.
val

val Epoch: 35. Iteration: 0. Loss: 1.3318341970443726. Accuracy: 43.
test Epoch: 35. Iteration: 0. Loss: 1.3318341970443726. Accuracy: 43.
val Epoch: 35. Iteration: 121. Loss: 1.3546011447906494. Accuracy: 43.
test Epoch: 35. Iteration: 121. Loss: 1.3546011447906494. Accuracy: 43.
val Epoch: 35. Iteration: 242. Loss: 1.3247289657592773. Accuracy: 43.
test Epoch: 35. Iteration: 242. Loss: 1.3247289657592773. Accuracy: 43.
val Epoch: 35. Iteration: 363. Loss: 1.3175629377365112. Accuracy: 43.
test Epoch: 35. Iteration: 363. Loss: 1.3175629377365112. Accuracy: 43.
val Epoch: 35. Iteration: 484. Loss: 1.3365792036056519. Accuracy: 43.
test Epoch: 35. Iteration: 484. Loss: 1.3365792036056519. Accuracy: 43.
val Epoch: 36. Iteration: 0. Loss: 1.3434083461761475. Accuracy: 43.
test Epoch: 36. Iteration: 0. Loss: 1.3434083461761475. Accuracy: 43.
val Epoch: 36. Iteration: 121. Loss: 1.3469691276550293. Accuracy: 43.
test Epoch: 36. Iteration: 121. Loss: 1.3469691276550293. Accuracy: 43.
val Epo

val Epoch: 46. Iteration: 363. Loss: 1.31023371219635. Accuracy: 43.
test Epoch: 46. Iteration: 363. Loss: 1.31023371219635. Accuracy: 43.
val Epoch: 46. Iteration: 484. Loss: 1.2808936834335327. Accuracy: 43.
test Epoch: 46. Iteration: 484. Loss: 1.2808936834335327. Accuracy: 43.
val Epoch: 47. Iteration: 0. Loss: 1.3229483366012573. Accuracy: 43.
test Epoch: 47. Iteration: 0. Loss: 1.3229483366012573. Accuracy: 43.
val Epoch: 47. Iteration: 121. Loss: 1.3129817247390747. Accuracy: 43.
test Epoch: 47. Iteration: 121. Loss: 1.3129817247390747. Accuracy: 43.
val Epoch: 47. Iteration: 242. Loss: 1.3262349367141724. Accuracy: 43.
test Epoch: 47. Iteration: 242. Loss: 1.3262349367141724. Accuracy: 43.
val Epoch: 47. Iteration: 363. Loss: 1.3154207468032837. Accuracy: 43.
test Epoch: 47. Iteration: 363. Loss: 1.3154207468032837. Accuracy: 43.
val Epoch: 47. Iteration: 484. Loss: 1.317575216293335. Accuracy: 43.
test Epoch: 47. Iteration: 484. Loss: 1.317575216293335. Accuracy: 43.
val Epoch

In [35]:
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

0:15:20
